<a href="https://colab.research.google.com/github/SreekarDugyala/Sreekar_INFO5731_Fall_2021/blob/main/In_class_exercise4_Sreekar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 11/03/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [ ]:
# Write your code here

import nltk
nltk.download('stopwords')




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pg0416\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:
pip install pyLDAvis==2.1.2

  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97737 sha256=1ecb9d9d261073dd11213f0add6334e290e880aca225295cf550478e47f42891
  Stored in directory: c:\users\pg0416\appdata\local\pip\cache\wheels\59\70\ee\99a0df99d8b4a7b87c79640ebee0927c0f6ccff046e9cc2471
Successfully built pyLDAvis
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install spacy

  Attempting uninstall: smart-open
    Found existing installation: smart-open 5.1.0
    Uninstalling smart-open-5.1.0:
      Successfully uninstalled smart-open-5.1.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#pip install spacy
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
#importing Packages
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
#import en_core_web_sm

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
#Preparing NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
# Importing Dataset
df = pd.read_csv('training.csv')

df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [ ]:
labels_dict = {0:'sadness',1:'joy',2:'love',3:'anger',4:'fear',5:'surprise'}
df['description'] = df['label'].map(labels_dict)
df.head

<bound method NDFrame.head of                                                     text  label description
0                                i didnt feel humiliated      0     sadness
1      i can go from feeling so hopeless to so damned...      0     sadness
2       im grabbing a minute to post i feel greedy wrong      3       anger
3      i am ever feeling nostalgic about the fireplac...      2        love
4                                   i am feeling grouchy      3       anger
...                                                  ...    ...         ...
15995  i just had a very brief time in the beanbag an...      0     sadness
15996  i am now turning and i feel pathetic that i am...      0     sadness
15997                     i feel strong and good overall      1         joy
15998  i feel like this was such a rude comment and i...      3       anger
15999  i know a lot but i feel so stupid because i ca...      0     sadness

[16000 rows x 3 columns]>

In [ ]:
# Converting to list
data = df.text.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

print(data[:1])

['i didnt feel humiliated']


In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['didnt', 'feel', 'humiliated']]


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['didnt', 'feel', 'humiliated']


In [ ]:
# Defining functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm", disable=['parser','ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['feel', 'humiliate']]


In [ ]:
# Creating Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1)]]


In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('feel', 1), ('humiliate', 1)]]

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.000*"mope" + 0.000*"onion" + 0.000*"touching" + 0.000*"conflicted" + '
  '0.000*"sentiment" + 0.000*"chamber" + 0.000*"ghost" + 0.000*"schindler" + '
  '0.000*"homosexual" + 0.000*"retreat"'),
 (1,
  '0.196*"people" + 0.150*"bit" + 0.101*"help" + 0.053*"glad" + 0.051*"week" + '
  '0.047*"ask" + 0.041*"move" + 0.033*"support" + 0.027*"torture" + '
  '0.022*"great"'),
 (2,
  '0.462*"make" + 0.097*"give" + 0.097*"s" + 0.077*"hate" + 0.039*"ill" + '
  '0.034*"unwelcome" + 0.011*"living" + 0.000*"even" + 0.000*"good" + '
  '0.000*"look"'),
 (3,
  '0.152*"ve" + 0.111*"take" + 0.100*"try" + 0.093*"also" + 0.078*"lot" + '
  '0.061*"keep" + 0.047*"alone" + 0.042*"point" + 0.038*"overwhelmed" + '
  '0.034*"important"'),
 (4,
  '0.395*"go" + 0.075*"home" + 0.067*"let" + 0.062*"care" + 0.053*"accept" + '
  '0.043*"sad" + 0.025*"easy" + 0.024*"hopeless" + 0.023*"hopeful" + '
  '0.017*"dissatisfied"'),
 (5,
  '0.826*"feel" + 0.034*"day" + 0.022*"pretty" + 0.022*"come" + 0.010*"part" + '
  

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -16.743437965199604

Coherence Score:  0.35995700464861546


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\pg0416\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.426281  0.220005       1        1  28.203947
15    -0.121845 -0.353241       2        1   7.860781
8     -0.074001 -0.080922       3        1   6.207054
7     -0.072003 -0.076009       4        1   6.154105
17    -0.057682 -0.048431       5        1   5.653908
6     -0.046130 -0.032657       6        1   5.360966
18    -0.028360 -0.012564       7        1   4.914617
3     -0.024468 -0.007963       8        1   4.840260
10    -0.001718  0.010384       9        1   4.503573
1      0.001228  0.011399      10        1   4.279193
12     0.041692  0.030189      11        1   3.633037
4      0.021379  0.022097      12        1   3.444016
2      0.032537  0.026527      13        1   3.309680
11     0.059142  0.036910      14        1   3.063179
19     0.065469  0.038394      15        1   2.753475
9      0.083648  0.041853      16        1   2.349015
14     0.103172  0.045016      17        1   1.943009
13     0.133668  0.046062      18        1   1.006020
16     0.147414  0.043921      19        1   0.484749
0      0.163139  0.039029      20        1   0.035414, topic_info=             Term          Freq         Total Category  logprob  loglift
0            feel  30834.000000  30834.000000  Default  30.0000  30.0000
10              m   4230.000000   4230.000000  Default  29.0000  29.0000
175           get   2089.000000   2089.000000  Default  28.0000  28.0000
43           make   2026.000000   2026.000000  Default  27.0000  27.0000
127       feeling   2120.000000   2120.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
6953     inherent      0.004478      1.492794  Topic20  -9.2556   2.1367
6954   additional      0.004478      1.492794  Topic20  -9.2556   2.1367
6955        sneak      0.004478      1.492794  Topic20  -9.2556   2.1367
6956        snoop      0.004478      1.492794  Topic20  -9.2556   2.1367
6957  indigestion      0.004478      1.492794  Topic20  -9.2556   2.1367

[896 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
125      19  0.986029        able
201      12  0.996176      accept
202      15  0.958935  acceptable
487      18  0.820009    accident
412       3  0.990296        ache
...     ...       ...         ...
82        2  0.995402       write
83        8  0.984660      writer
13        5  0.995948       wrong
48       11  0.997184        year
362      15  0.972139       young

[328 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 16, 9, 8, 18, 7, 19, 4, 11, 2, 13, 5, 3, 12, 20, 10, 15, 14, 17, 1])

In [ ]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!unzip mallet-2.0.8.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!pip install gensim

In [ ]:
from gensim import utils, matutils
from gensim.models import basemodel
#from gensim.models.wrappers import LdaMallet
from gensim.models.ldamodel import LdaModel
from gensim.utils import check_output, revdict

In [ ]:
# mallet_path = "/content/mallet-2.0.8/bin/mallet"
# ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
# Write your code here

import pandas as pd

# load data
df = pd.read_csv('training.csv')
df.head()


,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [ ]:
labels_dict = {0:'sadness',1:'joy',2:'love',3:'anger',4:'fear',5:'surprise'}
df['description'] = df['label'].map(labels_dict)
df.head

<bound method NDFrame.head of                                                     text  label description
0                                i didnt feel humiliated      0     sadness
1      i can go from feeling so hopeless to so damned...      0     sadness
2       im grabbing a minute to post i feel greedy wrong      3       anger
3      i am ever feeling nostalgic about the fireplac...      2        love
4                                   i am feeling grouchy      3       anger
...                                                  ...    ...         ...
15995  i just had a very brief time in the beanbag an...      0     sadness
15996  i am now turning and i feel pathetic that i am...      0     sadness
15997                     i feel strong and good overall      1         joy
15998  i feel like this was such a rude comment and i...      3       anger
15999  i know a lot but i feel so stupid because i ca...      0     sadness

[16000 rows x 3 columns]>

In [ ]:
print(df.shape)
df = df.dropna()
print(df.shape)

(16000, 3)
(16000, 3)


In [ ]:
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation \
                                        , preprocess_string, strip_short, stem_text

# preprocess given text
def preprocess(text):
    
    # clean text based on given filters
    CUSTOM_FILTERS = [lambda x: x.lower(), 
                                remove_stopwords, 
                                strip_punctuation, 
                                strip_short, 
                                stem_text]
    text = preprocess_string(text, CUSTOM_FILTERS)
    
    return text

# apply function to all reviews 
df['text (Clean)'] = df['text'].apply(lambda x: preprocess(x))

In [ ]:
from gensim import corpora

# create a dictionary with the corpus
corpus = df['text (Clean)']
dictionary = corpora.Dictionary(corpus)

# convert corpus into a bag of words
bow = [dictionary.doc2bow(text) for text in corpus]

In [ ]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# find the coherence score with a different number of topics
for i in range(2,11):
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=df['text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 2 clusters: 0.31062520094361
Coherence score with 3 clusters: 0.35230603413159073
Coherence score with 4 clusters: 0.32962553038414666
Coherence score with 5 clusters: 0.3322856258722176
Coherence score with 6 clusters: 0.3397158758082734
Coherence score with 7 clusters: 0.34254136279219066
Coherence score with 8 clusters: 0.3362658324751393
Coherence score with 9 clusters: 0.32511507565260106
Coherence score with 10 clusters: 0.3300720093802241


In [ ]:
lsi = LsiModel(bow, num_topics=10, id2word=dictionary)

for topic_num, words in lsi.print_topics(num_words=5):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.954*"feel" + 0.199*"like" + 0.057*"time" + 0.054*"know" + 0.051*"want".
Words in 1: 0.974*"like" + -0.204*"feel" + 0.025*"thing" + 0.020*"dont" + -0.019*"littl".
Words in 2: 0.640*"know" + 0.421*"time" + 0.367*"want" + 0.269*"love" + 0.223*"peopl".
Words in 3: 0.848*"time" + -0.334*"want" + -0.329*"know" + -0.125*"dont" + 0.095*"iv".
Words in 4: 0.841*"love" + -0.444*"know" + 0.184*"want" + 0.127*"peopl" + -0.095*"time".
Words in 5: 0.799*"want" + -0.414*"know" + -0.367*"love" + 0.165*"time" + -0.059*"dai".
Words in 6: 0.694*"littl" + 0.550*"dai" + 0.264*"thing" + 0.190*"bit" + 0.172*"iv".
Words in 7: -0.709*"think" + -0.370*"thing" + 0.328*"dai" + -0.308*"peopl" + 0.148*"love".
Words in 8: -0.693*"dai" + 0.593*"littl" + -0.193*"think" + 0.151*"bit" + -0.118*"peopl".
Words in 9: -0.828*"thing" + 0.401*"think" + 0.239*"littl" + 0.179*"peopl" + 0.126*"dai".


## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [ ]:
pip install lda2vec==0.16.10

  Created wheel for lda2vec: filename=lda2vec-0.16.10-py3-none-any.whl size=14434 sha256=a3641bf48bcc9e81ee406dca93e994b46c2aab552f1bf5758d092cd8a0aff9d7
  Stored in directory: c:\users\pg0416\appdata\local\pip\cache\wheels\fa\ad\6c\38aa944b34a94fd5d4f4d48e7432f94cd97f18d15779bdc9e5
Successfully built lda2vec
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Write your code here

from lda2vec import preprocess, Corpus
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

try:
    import seaborn
except:
    pass



ImportError: cannot import name 'preprocess' from 'lda2vec' (C:\Users\pg0416\Anaconda3\lib\site-packages\lda2vec\__init__.py)

In [ ]:
import pyLDAvis
pyLDAvis.enable_notebook()

In [ ]:
npz = np.load(open('topics.pyldavis.npz', 'r'))
dat = {k: v for (k, v) in npz.iteritems()}
dat['vocab'] = dat['vocab'].tolist()


top_n = 10
topic_to_topwords = {}
for j, topic_to_word in enumerate(dat['topic_term_dists']):
    top = np.argsort(topic_to_word)[::-1][:top_n]
    msg = 'Topic %i '  % j
    top_words = [dat['vocab'][i].strip()[:35] for i in top]
    msg += ' '.join(top_words)
    print (msg)
    topic_to_topwords[j] = top_words

FileNotFoundError: [Errno 2] No such file or directory: 'topics.pyldavis.npz'

In [ ]:
import warnings
warnings.filterwarnings('ignore')
prepared_data = pyLDAvis.prepare(dat['topic_term_dists'], dat['doc_topic_dists'], 
                                 dat['doc_lengths'] * 1.0, dat['vocab'], dat['term_frequency'] * 1.0, mds='tsne')

NameError: name 'dat' is not defined

In [ ]:
from sklearn.datasets import fetch_20newsgroups
remove=('headers', 'footers', 'quotes')
texts = fetch_20newsgroups(subset='train', remove=remove).data

In [ ]:
msg = "{weight:02d}% in topic {topic_id:02d} which has top words {text:s}"
for topic_id, weight in enumerate(dat['doc_topic_dists'][1]):
    if weight > 0.01:
        text = ', '.join(topic_to_topwords[topic_id])
        print msg.format(topic_id=topic_id, weight=int(weight * 100.0), text=text)

SyntaxError: invalid syntax (2646351304.py, line 5)

## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [ ]:
pip install google-colab

  Using cached google_colab-1.0.0-py2.py3-none-any.whl
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached portpicker-1.2.0-py3-none-any.whl
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached requests-2.21.0-py2.py3-none-any.whl (57 kB)
  Using cached prompt_toolkit-1.0.18-py3-none-any.whl (245 kB)
  Using cached simplegeneric-0.8.1-py3-none-any.whl
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
  Using cached urllib3-1.24.3-py2.py3-none-any.whl (118 kB)
  Running setup.py clean for pandas
Failed to build pandas
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.20
    Uninstalling prompt-toolkit-3.0.20:
      Successfully uninstalled prompt-toolkit-3.0.20
  Attempting uninstall:

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\pg0416\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\pg0416\\AppData\\Local\\Temp\\pip-install-rq473qhg\\pandas_d54155ddf18549cebbc980afa3b068e5\\setup.py'"'"'; __file__='"'"'C:\\Users\\pg0416\\AppData\\Local\\Temp\\pip-install-rq473qhg\\pandas_d54155ddf18549cebbc980afa3b068e5\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\pg0416\AppData\Local\Temp\pip-wheel-awmkwfqi'
       cwd: C:\Users\pg0416\AppData\Local\Temp\pip-install-rq473qhg\pandas_d54155ddf18549cebbc980afa3b068e5\
  Complete output (843 lines):
  C:\Users\pg0416\AppData\Local\Temp\pip-install-rq473qhg\pandas_d54155ddf18549cebbc980afa3b068e5\setup.py:50: 

  copying pandas\tests\io\msgpack\test_newspec.py -> build\lib.win-amd64-3.9\pandas\tests\io\msgpack
  copying pandas\tests\io\msgpack\test_obj.py -> build\lib.win-amd64-3.9\pandas\tests\io\msgpack
  copying pandas\tests\io\msgpack\test_pack.py -> build\lib.win-amd64-3.9\pandas\tests\io\msgpack
  copying pandas\tests\io\msgpack\test_read_size.py -> build\lib.win-amd64-3.9\pandas\tests\io\msgpack
  copying pandas\tests\io\msgpack\test_seq.py -> build\lib.win-amd64-3.9\pandas\tests\io\msgpack
  copying pandas\tests\io\msgpack\test_sequnpack.py -> build\lib.win-amd64-3.9\pandas\tests\io\msgpack
  copying pandas\tests\io\msgpack\test_subtype.py -> build\lib.win-amd64-3.9\pandas\tests\io\msgpack
  copying pandas\tests\io\msgpack\test_unpack.py -> build\lib.win-amd64-3.9\pandas\tests\io\msgpack
  copying pandas\tests\io\msgpack\test_unpack_raw.py -> build\lib.win-amd64-3.9\pandas\tests\io\msgpack
  copying pandas\tests\io\msgpack\__init__.py -> build\lib.win-amd64-3.9\pandas\tests\io\msgpack

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('/content/gdrive/My Drive/Assignment/tokyo_2020_tweets.csv')
df = df.dropna()
df = df.sample(20000, random_state=42)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/My Drive/Assignment/tokyo_2020_tweets.csv'

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

def clean_text(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'#[A-Za-z0-9]*', ' ', x)
  x = re.sub(r'https*://.*', ' ', x)
  x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
  tokens = word_tokenize(x)
  x = ' '.join([w for w in tokens if not w.lower() in stop_words])
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
  x = re.sub(r'\d+', ' ', x)
  x = re.sub(r'\n+', ' ', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x


df['clean_text'] = df.text.apply(clean_text)
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pg0416\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pg0416\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


,text,label,description,text (Clean),clean_text
0,i didnt feel humiliated,0,sadness,"[didnt, feel, humili]",didnt feel humiliated
1,i can go from feeling so hopeless to so damned...,0,sadness,"[feel, hopeless, damn, hope, care, awak]",go feeling hopeless damned hopeful around some...
2,im grabbing a minute to post i feel greedy wrong,3,anger,"[grab, minut, post, feel, greedi, wrong]",im grabbing minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...,2,love,"[feel, nostalg, fireplac, know, properti]",ever feeling nostalgic fireplace know still pr...
4,i am feeling grouchy,3,anger,"[feel, grouchi]",feeling grouchy


In [ ]:
! pip install bertopic

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\pg0416\Anaconda3\python.exe' 'C:\Users\pg0416\Anaconda3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' build_wheel 'C:\Users\pg0416\AppData\Local\Temp\tmpjd5kauwf'
       cwd: C:\Users\pg0416\AppData\Local\Temp\pip-install-hgfnp0mf\hdbscan_1300dce1ac794f4fb99bd1c52b6cb95f
  Complete output (40 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-39
  creating build\lib.win-amd64-cpython-39\hdbscan
  copying hdbscan\flat.py -> build\lib.win-amd64-cpython-39\hdbscan
  copying hdbscan\hdbscan_.py -> build\lib.win-amd64-cpython-39\hdbscan
  copying hdbscan\plots.py -> build\lib.win-amd64-cpython-39\hdbscan
  copying hdbscan\prediction.py -> build\lib.win-amd64-cpython-39\hdbscan
  copying hdbscan\robust_single_linkage_.py -> build\lib.win-amd64-cpython-39\hdbscan
  copying hdbscan\validity.py -> build\lib.win-amd64-cpython-39\hdbscan
  


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125940 sha256=56b127331793bda1968dd56814b2f01e6cb459ae27964ad8de9d794797d90139
  Stored in directory: c:\users\pg0416\appdata\local\pip\cache\wheels\71\67\06\162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=a10f6e90c21513bc76cb3583db732fee27573f34adfd79cbf39a006fb33e0cf3
  Stored in directory: c:\users\pg0416\appdata\local\pip\cache\wheels\f4\3e\1c\596d0a463d17475af648688443fa4846fef624d1390339e7e9
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55512 

In [ ]:
from bertopic import BERTopic
tweets = df.clean_text.to_list()
timestamp = df.date.to_list()

topic_model = BERTopic(language="english")
topics, probs = topic_model.fit_transform(tweets)

topic_model.get_topic_info()

ModuleNotFoundError: No module named 'bertopic'

In [ ]:
topic_model.get_topic_info(0)

NameError: name 'topic_model' is not defined

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(top_n_topics=12, n_words = 10, width = 350, height = 350)

## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
# Write your answer here (no code needed for this question)

"""
The results are similar.
"""
#which is better between lda,lsa?
"""
LSA.

in Natural Language Processing (NLP): Latent Semantic Analysis (LSA), 
Latent Dirichlet Allocation (LDA) and lexical chains. These techniques were evaluated 
and compared on two different corpora in order to highlight the similarities and differences 
between them from a semantic analysis viewpoint. The first corpus consisted of four Wikipedia 
articles on different topics, while the second one consisted of 35 online chat conversations 
between 4-12 participants debating four imposed topics (forum, chat, blog and wikis). The study 
focuses on finding similarities and differences between the outcomes of the three methods from a 
semantic analysis point of view, by computing quantitative factors such as correlations, degree of 
coverage of the resulting topics, etc. Using corpora from different types of discourse and quantitative 
factors that are task-independent allows us to prove that although LSA and LDA provide similar results,
the results of lexical chaining are not very correlated with neither the ones of LSA or LDA, therefore 
lexical chains might be used complementary to LSA or LDA when performing semantic analysis for various NLP applications.
These factors out to be the reason why the LSA are the better.
"""



'\nLSA.\n\nin Natural Language Processing (NLP): Latent Semantic Analysis (LSA), \nLatent Dirichlet Allocation (LDA) and lexical chains. These techniques were evaluated \nand compared on two different corpora in order to highlight the similarities and differences \nbetween them from a semantic analysis viewpoint. The first corpus consisted of four Wikipedia \narticles on different topics, while the second one consisted of 35 online chat conversations \nbetween 4-12 participants debating four imposed topics (forum, chat, blog and wikis). The study \nfocuses on finding similarities and differences between the outcomes of the three methods from a \nsemantic analysis point of view, by computing quantitative factors such as correlations, degree of \ncoverage of the resulting topics, etc. Using corpora from different types of discourse and quantitative \nfactors that are task-independent allows us to prove that although LSA and LDA provide similar results,\nthe results of lexical chaining ar